# Clash Royale Data Visualization Dashboard
## By: Atharv Patwardhan
## RUID: 226007305

# Introduction

In this notebook, we will be analyzing a dataset containig data about the mobile game 'Clash Royale'. Clash Royale is a real-time multiplayer strategy game developed by Supercell, combining elements of tower defense, card collection, and competitive gameplay. Players engage in battles using a deck of cards representing troops, spells, and buildings to outsmart their opponents and destroy enemy towers. Each card costs a certain amount of elixir, which is kind of like the currency during battle. I have been playing clash royale since a couple of years, hence this dataset appealed to me while I was searching for a dataset during my final project.

The project incorporates multivariate visualizations using colors, markers and also uses machine learning models like K Means to make clusters between data points. All these combined with the interactive plots are aimed to give the viewer insights which would eventually help them in making their own strategies while playing the game.

The data for this project is taken from this data source: https://www.kaggle.com/datasets/bwandowando/clash-royale-season-18-dec-0320-dataset/data


# Motivation

The motivation behind this project is to help Clash Royale players make better game plans based on the insights given in this report.
We look at insights for each arena and look at key factors such as elixir costs, card rarity, and spells. All these factors are the most discussed ones amongst the Clash Royale Community. Over the years I spent playing this game, I learned what matters the most to players and how players can improve their gameplay. I tried to incorporate this experience as much as I could while deciding on what metrics and insights to provide!

# Methods


## Importing modules

In [276]:
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import opts
import panel as pn
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

pn.extension()
hv.extension("bokeh")

## Data Loading and Cleaning

Please Note: I was having trouble loading this csv file in codespaces due to it's size but it was running fine locally.

In [278]:
battle_data = pd.read_csv("data/BattlesStaging_01012021_WL_tagged.csv")

In [280]:
battle_data.head()

,Unnamed: 0,battleTime,arena.id,gameMode.id,average.startingTrophies,winner.tag,winner.startingTrophies,winner.trophyChange,winner.crowns,winner.kingTowerHitPoints,...,loser.cards.list,loser.totalcard.level,loser.troop.count,loser.structure.count,loser.spell.count,loser.common.count,loser.rare.count,loser.epic.count,loser.legendary.count,loser.elixir.average
0,0,2020-12-31 21:02:12+00:00,54000050.0,72000006.0,5363.0,#PVLPJP2Y,5372.0,28.0,2.0,4145.0,...,"[26000000, 26000015, 26000023, 27000004, 28000...",104,3,1,4,1,1,4,2,3.500
1,1,2020-12-31 21:02:15+00:00,54000050.0,72000006.0,5407.0,#8PRLRYYCV,5409.0,29.0,1.0,5304.0,...,"[26000023, 26000027, 26000037, 26000046, 26000...",104,6,1,1,0,1,2,5,4.250
2,2,2020-12-31 21:02:45+00:00,54000050.0,72000006.0,5741.0,#2G8LQRCG,5749.0,28.0,2.0,5762.0,...,"[26000022, 26000027, 26000028, 26000041, 26000...",104,7,0,1,4,2,1,1,4.125
3,3,2020-12-31 21:03:13+00:00,54000050.0,72000006.0,4307.0,#Y9QL09VGV,4316.0,28.0,2.0,4392.0,...,"[26000012, 26000027, 26000031, 26000033, 26000...",80,6,1,1,2,1,2,3,3.750
4,4,2020-12-31 21:03:17+00:00,54000050.0,72000006.0,5776.5,#9RRYG9P9U,5783.0,28.0,3.0,5832.0,...,"[26000010, 26000011, 26000021, 26000037, 26000...",104,5,1,2,2,4,0,2,3.250


As you can see above, the data is not very intuitive at first. The data points are not organized and it is very hard to derive any meaning from it. To solve this, I placed these datapoints into arenas. Arenas are like leagues in Clash Royale. As you get better and beat more players, you advance into higher arenas. I used the same logic the game uses to put players into arenas. The trophy ranges in the below code are the exact ranges used in the game!

### Assigning arenas to each entry in the dataset

In [282]:
trophies = [
    0,
    300,
    600,
    1000,
    1300,
    1600,
    2000,
    2300,
    2600,
    3000,
    3400,
    3800,
    4200,
    4600,
    5000,
    5500,
    6000,
    6500,
    7000,
    7500,
    8000,
    8500,
    9000,
    float("inf"),
]
arenas = [
    "1: Goblin Stadium",
    "2: Bone Pit",
    "3: Barbarian Bowl",
    "4: Spell Valley",
    "5: Builder's Workshop",
    "6: P.E.K.K.A's Playhouse",
    "7: Royal Arena",
    "8: Frozen Peak",
    "9: Jungle Arena",
    "10: Hog Mountain",
    "11: Electro Valley",
    "12: Spooky Town",
    "13: Rascal's Hideout",
    "14: Serenity Peak",
    "15: Miner's Mine",
    "16: Executioner's Kitchen",
    "17: Royal Crypt",
    "18: Silent Sanctuary",
    "19: Dragon Spa",
    "20: Boot Camp",
    "21: Clash Fest",
    "22: PANCAKES!",
    "23: Legendary Arena",
]

battle_data["average.startingTrophies"] = battle_data[
    "average.startingTrophies"
].fillna(0)

# assign the arena names based on the trophy ranges
battle_data["arena.name"] = pd.cut(
    battle_data["average.startingTrophies"], bins=trophies, labels=arenas, right=False
)

Now that each entry is in an arena, we can start working with the data.

I am also defining the arena selector used in the dashboard over here, as it is used in the plotting functions below.

In [284]:
arenas = battle_data["arena.name"].unique().tolist()

arenas.sort()
arena_selector = pn.widgets.Select(name="Arena", options=arenas)

First, we will define a function to look at card usage for each Arena. Cards are divided into four categories of rarity: Common, Rare, Epic, and Legendary.

In [286]:
def get_card_usage(arena):
    filtered_battle_data = battle_data[battle_data["arena.name"] == arena]

    # card counts for winners and losers by card rarity
    winner_card_counts = {
        "Common": filtered_battle_data["winner.common.count"].sum(),
        "Rare": filtered_battle_data["winner.rare.count"].sum(),
        "Epic": filtered_battle_data["winner.epic.count"].sum(),
        "Legendary": filtered_battle_data["winner.legendary.count"].sum(),
    }

    loser_card_counts = {
        "Common": filtered_battle_data["loser.common.count"].sum(),
        "Rare": filtered_battle_data["loser.rare.count"].sum(),
        "Epic": filtered_battle_data["loser.epic.count"].sum(),
        "Legendary": filtered_battle_data["loser.legendary.count"].sum(),
    }

    data = {
        "Card Type": list(winner_card_counts.keys()) * 2,
        "Count": list(winner_card_counts.values()) + list(loser_card_counts.values()),
        "Category": ["Winner"] * 4 + ["Loser"] * 4,
    }

    return pd.DataFrame(data)

The function below plots a bar chart for card usage using the get_card_usage function defined above.

In [288]:
@pn.depends(arena_selector)
def update_plot(arena):
    plot_battle_data = get_card_usage(arena)
    bars = hv.Bars(plot_battle_data, ["Card Type", "Category"], "Count").opts(
        opts.Bars(
            width=700,
            height=400,
            title=f"Card Usage by Category in {arena}",
            xlabel="Card Type",
            ylabel="Count",
            legend_position="top_right",
            tools=["hover"],
            color=hv.dim("Category").categorize({"Winner": "blue", "Loser": "red"}),
            show_legend=True,
        )
    )
    return bars

Each player has a deck of 8 cards. In the below code, we will be taking the 8 cards for each winner and loser, and then calculating the grand totals, to see which cards are the most used and popular.

Citation: The below code section for creating the card deck dataset takes inspiration from a kaggle source.

In [290]:
battle_data_winner_card1 = battle_data[["arena.name", "winner.card1.id"]].pivot_table(
    columns="winner.card1.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_winner_card2 = battle_data[["arena.name", "winner.card2.id"]].pivot_table(
    columns="winner.card2.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_winner_card3 = battle_data[["arena.name", "winner.card3.id"]].pivot_table(
    columns="winner.card3.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_winner_card4 = battle_data[["arena.name", "winner.card4.id"]].pivot_table(
    columns="winner.card4.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_winner_card5 = battle_data[["arena.name", "winner.card5.id"]].pivot_table(
    columns="winner.card5.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_winner_card6 = battle_data[["arena.name", "winner.card6.id"]].pivot_table(
    columns="winner.card6.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_winner_card7 = battle_data[["arena.name", "winner.card7.id"]].pivot_table(
    columns="winner.card7.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_winner_card8 = battle_data[["arena.name", "winner.card8.id"]].pivot_table(
    columns="winner.card8.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)

In [292]:
battle_data_loser_card1 = battle_data[["arena.name", "loser.card1.id"]].pivot_table(
    columns="loser.card1.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_loser_card2 = battle_data[["arena.name", "loser.card2.id"]].pivot_table(
    columns="loser.card2.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_loser_card3 = battle_data[["arena.name", "loser.card3.id"]].pivot_table(
    columns="loser.card3.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_loser_card4 = battle_data[["arena.name", "loser.card4.id"]].pivot_table(
    columns="loser.card4.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_loser_card5 = battle_data[["arena.name", "loser.card5.id"]].pivot_table(
    columns="loser.card5.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_loser_card6 = battle_data[["arena.name", "loser.card6.id"]].pivot_table(
    columns="loser.card6.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_loser_card7 = battle_data[["arena.name", "loser.card7.id"]].pivot_table(
    columns="loser.card7.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)
battle_data_loser_card8 = battle_data[["arena.name", "loser.card8.id"]].pivot_table(
    columns="loser.card8.id",
    index=["arena.name"],
    aggfunc=len,
    fill_value=0,
    observed=False,
)

In [294]:
battle_data_winner_combined = (
    battle_data_winner_card1
    + battle_data_winner_card2
    + battle_data_winner_card3
    + battle_data_winner_card4
    + battle_data_winner_card5
    + battle_data_winner_card6
    + battle_data_winner_card7
    + battle_data_winner_card8
)
battle_data_loser_combined = (
    battle_data_loser_card1
    + battle_data_loser_card2
    + battle_data_loser_card3
    + battle_data_loser_card4
    + battle_data_loser_card5
    + battle_data_loser_card6
    + battle_data_loser_card7
    + battle_data_loser_card8
)
battle_data_grand_totals = battle_data_winner_combined + battle_data_loser_combined

In [296]:
battle_data_grand_totals.head()

winner.card1.id,26000000,26000001,26000002,26000003,26000004,26000005,26000006,26000007,26000008,26000009,...,28000009,28000010,28000011,28000012,28000013,28000014,28000015,28000016,28000017,28000018
arena.name,,,,,,,,,,,,,,,,,,,,,
1: Goblin Stadium,4777,3537,937,5945,0,4273,0,35,0,0,...,0,0,0,0,0,0,0,1,0,1
2: Bone Pit,3201,2374,455,4998,0,2922,2,1846,4,3,...,0,0,0,0,0,0,1,0,0,0
3: Barbarian Bowl,3557,3034,545,4374,6,3241,21,5803,2208,1641,...,0,0,0,3,2,0,863,7,0,3
4: Spell Valley,3617,2996,673,3984,2492,2880,139,10605,3279,3526,...,1,1,13,4,10,1,2125,8,9,6
5: Builder's Workshop,2644,2887,727,2872,2262,2130,71,4004,758,697,...,202,3,5,247,0,0,806,1,0,4


In [298]:
battle_data_grand_totals = battle_data_grand_totals.reset_index()
battle_data_loser_combined = battle_data_loser_combined.reset_index()
battle_data_winner_combined = battle_data_winner_combined.reset_index()

This is where the second dataset, the CardMasterList comes in. It contains the mapping of each card name with it's corresponding card ID. We use this dataset to rename the columns in the above dataset.

In [300]:
battle_data_cardmasterlist = pd.read_csv("data/CardMasterListSeason18_12082020.csv")
battle_data_cardmasterlist.reset_index(inplace=True)
cardmasterlist_dict = dict(
    zip(
        battle_data_cardmasterlist["team.card1.id"],
        battle_data_cardmasterlist["team.card1.name"],
    )
)

In [302]:
battle_data_grand_totals = battle_data_grand_totals.rename(columns=cardmasterlist_dict)
battle_data_loser_combined = battle_data_loser_combined.rename(
    columns=cardmasterlist_dict
)
battle_data_winner_combined = battle_data_winner_combined.rename(
    columns=cardmasterlist_dict
)

In [304]:
battle_data_grand_totals.columns = battle_data_grand_totals.columns.str.strip()
battle_data_winner_combined["arena.name"] = battle_data_winner_combined[
    "arena.name"
].str.strip()
battle_data_loser_combined["arena.name"] = battle_data_loser_combined[
    "arena.name"
].str.strip()

battle_data_winner_combined.set_index("arena.name")
battle_data_winner_combined.set_index("arena.name", inplace=True)
battle_data_loser_combined.set_index("arena.name", inplace=True)

In [306]:
battle_data_winner_combined.head()

winner.card1.id,Knight,Archers,Goblins,Giant,P.E.K.K.A,Minions,Balloon,Witch,Barbarians,Golem,...,Poison,Graveyard,The Log,Tornado,Clone,Earthquake,Barbarian Barrel,Heal Spirit,Giant Snowball,Royal Delivery
arena.name,,,,,,,,,,,,,,,,,,,,,
1: Goblin Stadium,2264,1649,446,2742,0,2077,0,26,0,0,...,0,0,0,0,0,0,0,0,0,0
2: Bone Pit,1247,865,223,1974,0,1285,1,1023,3,2,...,0,0,0,0,0,0,1,0,0,0
3: Barbarian Bowl,1077,867,244,1329,4,1372,15,3453,1084,815,...,0,0,0,3,1,0,528,3,0,2
4: Spell Valley,1316,1027,311,1353,1205,1319,87,5800,1578,1707,...,1,1,6,2,4,1,1243,2,5,1
5: Builder's Workshop,683,565,140,504,1365,589,46,2572,420,417,...,121,2,3,134,0,0,587,0,0,2


Now the dataset has the grand totals of each card for each arena. This kaggle source was incredibly useful as it allowed me to obtain this dataset which contains card usage for each arena!

# Looking at Top Cards for each Arena

In [308]:
@pn.depends(arena_selector)
def get_top_cards_by_arena(arena):
    top_cards = battle_data_grand_totals[
        battle_data_grand_totals["arena.name"] == arena
    ]
    card_usage = top_cards.drop(columns=["arena.name"])
    total_usage = card_usage.sum().sort_values(ascending=False)
    top_10_cards = total_usage.head(10)

    bar_plot = hv.Bars(top_10_cards)

    bar_plot.opts(
        title=f"Top 10 Most Used Cards in Arena {arena}",
        xlabel="Card",
        ylabel="Usage",
        xrotation=45,
        width=800,
        height=400,
        color="blue",
    )

    return bar_plot

## Diving into Elixir

First, I created a new dataframe for the elixir data. I added fields like elixir mean, mode, median and loser and winner means to this dataframe.

In [310]:
def get_average_elixir(arena):
    print(arena)
    filtered_battle_data = battle_data[battle_data["arena.name"] == arena]
    data = {
        "winner_mean_elixir": filtered_battle_data["winner.elixir.average"].mean(),
        "winner_elixir_mode": filtered_battle_data["winner.elixir.average"].mode(),
        "winner_elixir_median": filtered_battle_data["winner.elixir.average"].median(),
        "loser_mean_elixir": filtered_battle_data["loser.elixir.average"].mean(),
        "loser_elixir_mode": filtered_battle_data["loser.elixir.average"].mode(),
        "loser_elixir_median": filtered_battle_data["loser.elixir.average"].median(),
        "winner_higher_elixir": (
            filtered_battle_data["loser.elixir.average"]
            < filtered_battle_data["winner.elixir.average"]
        ).sum(),
        "loser_higher_elixir": (
            filtered_battle_data["loser.elixir.average"]
            > filtered_battle_data["winner.elixir.average"]
        ).sum(),
    }

    return pd.DataFrame(data)

In [312]:
avg_elixir_dataframes = []

for i in battle_data["arena.name"].unique():
    arena_data = get_average_elixir(i)
    arena_data["arena.name"] = i
    avg_elixir_dataframes.append(arena_data)

avg_elixir_data = pd.concat(avg_elixir_dataframes, ignore_index=True)

15: Miner's Mine
16: Executioner's Kitchen
13: Rascal's Hideout
14: Serenity Peak
12: Spooky Town
17: Royal Crypt
18: Silent Sanctuary
19: Dragon Spa
11: Electro Valley
9: Jungle Arena
3: Barbarian Bowl
10: Hog Mountain
6: P.E.K.K.A's Playhouse
7: Royal Arena
4: Spell Valley
1: Goblin Stadium
2: Bone Pit
8: Frozen Peak
5: Builder's Workshop


In [314]:
avg_elixir_data.head()

,winner_mean_elixir,winner_elixir_mode,winner_elixir_median,loser_mean_elixir,loser_elixir_mode,loser_elixir_median,winner_higher_elixir,loser_higher_elixir,arena.name
0,3.781931,3.875,3.875,3.784882,3.875,3.875,273097,276825,15: Miner's Mine
1,3.713890,3.875,3.750,3.701368,3.875,3.750,123152,121028,16: Executioner's Kitchen
2,3.904017,3.875,3.875,3.899913,3.875,3.875,324370,323714,13: Rascal's Hideout
3,3.900592,3.875,3.875,3.906016,3.875,3.875,360220,366335,14: Serenity Peak
4,3.937653,3.875,3.875,3.928495,3.875,3.875,127607,126523,12: Spooky Town


Now that the elixir data is in a proper dataframe, it makes it much more convenient to plot.

## Elixir data by Arena

Below is a function is to get the mean elixir for both winners and losers for each arena.

In [316]:
@pn.depends(arena_selector)
def get_elixir_data_by_arena(arena):
    selected_arena_avg_elixir = avg_elixir_data[avg_elixir_data["arena.name"] == arena]

    bars = selected_arena_avg_elixir.hvplot.bar(
        y=["winner_mean_elixir", "loser_mean_elixir"],
        x="arena.name",
        title=f"Elixir Means for Arena: {arena}",
        ylabel="Elixir Mean",
        xlabel="Arena",
        legend=True,
        color=["blue", "red"],
        height=400,
        width=600,
    )

    return bars

Getting elixir metrics for arenas.

In [318]:
@pn.depends(arena_selector)
def elixir_metrics_for_one_arena(arena):
    selected_arena_avg_elixir = avg_elixir_data[avg_elixir_data["arena.name"] == arena]
    most_recurring = selected_arena_avg_elixir["winner_elixir_mode"].mode()
    median_elixir = selected_arena_avg_elixir["winner_elixir_median"].median()
    mean_elixir = selected_arena_avg_elixir["winner_mean_elixir"].mean()

    return pn.Column(
        f"## **Most popular average elixir:** {most_recurring.iloc[0]}",
        f"## **Mean elixir average:** {mean_elixir:.2f}",
    )

The function below creates a line chart showing how average elixir changes by arena.

In [320]:
def get_elixir_metrics_by_arena():
    grouped = avg_elixir_data.groupby("arena.name").mean()
    line_chart = grouped.hvplot.line(
        y=["winner_mean_elixir", "loser_mean_elixir"],
        xlabel="Arena",
        ylabel="Elixir",
        title="How average elixir changes by arena",
        legend=True,
        height=400,
        width=700,
        ylim=(3.0, 5.0),
    ).opts(xrotation=90)

    return line_chart

## Using K Means Clustering to make clusters in the elixir scatterplot.

### I used K Means to make 4 clusters:
1. Winner elixir average is high and loser elixir average is low.
2. Loser elixir average is high and winner elixir average is low.
3. Both winner and loser elixir averages are low.
4. Both winner and loser elixir averages are high.

The goal was to compare the distribution of points in these clusters across arenas, hopefully to find different concentrations of points in the clusters in different arenas.

In [322]:
@pn.depends(arena_selector)
def elixir_clusters(arena):
    filtered_data = battle_data[battle_data["arena.name"] == arena][
        ["winner.elixir.average", "loser.elixir.average"]
    ]
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(filtered_data)

    kmeans = KMeans(n_clusters=4, random_state=1)
    filtered_data["Cluster"] = kmeans.fit_predict(scaled_data)

    scatter = hv.Scatter(
        filtered_data,
        kdims=["winner.elixir.average"],
        vdims=["loser.elixir.average", "Cluster"],
    ).opts(
        color="Cluster",
        cmap="Category10",
        size=8,
        alpha=0.7,
        xlabel="Winner Elixir Average",
        ylabel="Loser Elixir Average",
        title=f"Clusters of Players in {arena}",
        legend_position="right",
        height=600,
        width=600,
    )
    return scatter.opts(tools=["hover"], show_legend=True)

## Do Spells Matter?

Here I am asking the question if spells actually make a difference during battle. Personally, I do not use spells during battle and I do not think spells are useful but I wanted to see if the insights provided by the plots agree with my understanding.

In [324]:
@pn.depends(arena_selector)
def spells(arena):
    filtered_battle_data = battle_data[battle_data["arena.name"] == arena]
    count_w = (
        filtered_battle_data["winner.spell.count"] > 0
    ).sum()  # this is the winner spell count
    count_wl = (
        (filtered_battle_data["winner.spell.count"] > 0)
        & (filtered_battle_data["loser.spell.count"] == 0)
    ).sum()  # this is the count where winner used spells but loser did not
    count_lw = (
        (filtered_battle_data["winner.spell.count"] == 0)
        & (filtered_battle_data["loser.spell.count"] > 0)
    ).sum()  # this is the count of battles where loser used spells but winners did not
    count_l = (
        filtered_battle_data["loser.spell.count"] > 0
    ).sum()  # this is the loser spell count

    return pn.Column(
        f"## **Mean spell count:** {filtered_battle_data['winner.spell.count'].mean()}",
        f"## **Ratio of winners using spells:** {count_w/len(filtered_battle_data)}",
        f"## **Used only by winner:** {count_wl}",
        f"## **Used only by loser:** {count_lw}",
        f"## **Ratio of losers using spells:** {count_l/len(filtered_battle_data)}",
    )

In [326]:
@pn.depends(arena_selector)
def spell_bar_chart(arena):
    filtered_battle_data = battle_data[battle_data["arena.name"] == arena]
    winner_mean_spell_count = filtered_battle_data["winner.spell.count"].mean()
    loser_mean_spell_count = filtered_battle_data["loser.spell.count"].mean()

    new_spells_df = pd.DataFrame(
        {
            "Title": ["Winner Mean Spell Count", "Loser Mean Spell Count"],
            "Spell means": [winner_mean_spell_count, loser_mean_spell_count],
        }
    )

    spell_bar_chart = hv.Bars(new_spells_df, kdims=["Title"], vdims=["Spell means"])
    spell_bar_chart.opts(
        title=f"Mean Spell Counts for Arena: {arena}",
        xlabel="Winner/Loser",
        ylabel="Mean Spell Count",
        legend_position="top_right",
        tools=["hover"],
        color=hv.dim("Title").categorize(
            {"Winner Mean Spell Count": "blue", "Loser Mean Spell Count": "red"}
        ),
        show_legend=True,
        height=400,
        width=600,
    )

    return spell_bar_chart

## Crowns Analysis
During each match, players win crowns by destroying their opponent's towers. You get one town for each rincess tower you destroy. Destroying the king tower gets 3 crowns and you win the match. 

In [328]:
@pn.depends(arena_selector)
def get_crown_bar_plot(arena):
    filtered_crown_data = battle_data[battle_data["arena.name"] == arena][
        ["winner.crowns", "loser.crowns"]
    ]

    winner_mean_crowns = filtered_crown_data["winner.crowns"].mean()
    loser_mean_crowns = filtered_crown_data["loser.crowns"].mean()

    crowns_data = pd.DataFrame(
        {
            "Category": ["Winner Crowns", "Loser Crowns"],
            "Mean Crowns": [winner_mean_crowns, loser_mean_crowns],
        }
    )

    crown_bar_plot = hv.Bars(
        crowns_data, kdims=["Category"], vdims=["Mean Crowns"]
    ).opts(
        title=f"Mean Crowns for Arena: {arena}",
        xlabel="Category",
        ylabel="Mean Crowns",
        tools=["hover"],
        color=hv.dim("Category").categorize(
            {"Winner Crowns": "blue", "Loser Crowns": "red"}
        ),
        height=400,
        width=600,
        legend_position="top_right",
        show_legend=False,
    )

    return crown_bar_plot

# The Dashboard: Where it all comes together!

In [330]:
class Dashboard:
    def __init__(self):

        self.title = pn.pane.Markdown("# Clash Royale Interactive Dashboard")

        self.intro = (
            "Introduction",
            pn.Column(
                pn.pane.Markdown("# Introduction"),
                "Clash Royale is a real-time multiplayer strategy game developed by Supercell, combining elements of tower defense, card collection, and competitive gameplay. Players engage in battles using a deck of cards representing troops, spells, and buildings to outsmart their opponents and destroy enemy towers.",
                "## In this dashboard we will do: \n 1. Card Usage Analysis \n 2. Elixir Analysis \n 3. Spells Analysis \n 4. Crowns Analysis",
            ),
        )

        self.layout1 = (
            "Cards",
            pn.Column(
                arena_selector,
                pn.pane.Markdown(
                    "### We will look at the top 10 cards and the card usage distribution by rarity across different arenas in this section."
                ),
                update_plot,
                get_top_cards_by_arena,
            ),
        )

        self.layout2 = (
            "Looking into Elixir",
            pn.Column(
                arena_selector,
                pn.pane.Markdown(
                    "### In this section, we will look at elixir trends across different arenas."
                ),
                pn.Row(get_elixir_data_by_arena, get_elixir_metrics_by_arena),
                pn.pane.Markdown(
                    "## Scatterplot of winner and loser average elixir clusters."
                ),
                pn.Row(
                    elixir_clusters,
                    "## The points are clustered into four categories: \n ### Cluster 0 (Dark Blue): Both Winner and loser elixir averages are high.\n ### Cluster 1 (Red): Both winner and loser elixir averages are low. \n ### Cluster 2 (Pink): Loser elixir average is high and winner elixir average is low. \n ### Cluster 3 (Green): Winner elixir average is high and loser elixir average is low. \n ### As a general threshold, anything below 3.5 is considered to be a low average, and anything above 4.0 is considered a high elixir average. 3.5 - 4.0 is considered the optimal average for a deck.",
                ),
                elixir_metrics_for_one_arena,
            ),
        )
        self.layout3 = (
            "Do Spells matter?",
            pn.Column(
                arena_selector,
                pn.pane.Markdown(
                    "### In this section we will see if using spells actually helps and by how much, across different arenas."
                ),
                pn.Row(spell_bar_chart, spells),
            ),
        )
        self.layout4 = (
            "Crown Analysis",
            pn.Column(
                arena_selector,
                pn.Row(get_crown_bar_plot),
                pn.Row(
                    "## In a Clash Royale Match, a player can obtain upto three crowns by destroying the opponents tower. The player with the most number of crowns wins at the end of the match."
                ),
                pn.Row(
                    "## As we can observe, the average number of crowns per winner reduce as the arena level rises. This is because it becomes harder to get crowns in later arenas."
                ),
            ),
        )

        self.layout = pn.Tabs(
            self.intro, self.layout1, self.layout2, self.layout3, self.layout4
        )

    def show(self):
        return pn.Column(self.title, self.layout)

In [332]:
d = Dashboard()
d.show()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] Markdown(str)
        [1] Column
            [0] Select(name='Arena', options=['10: Hog Mountain', ...], value='10: Hog Mountain')
            [1] Markdown(str)
            [2] ParamFunction(function, _pane=HoloViews, defer_load=False)
            [3] ParamFunction(function, _pane=HoloViews, defer_load=False)
        [2] Column
            [0] Select(name='Arena', options=['10: Hog Mountain', ...], value='10: Hog Mountain')
            [1] Markdown(str)
            [2] Row
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
                [1] Column(sizing_mode='fixed')
                    [0] Column()
                    [1] Row(sizing_mode='fixed')
                        [0] HoloViews(NdOverlay, height=400, name='interactive29186', sizing_mode='fixed', width=700)
            [3] Markdown(str)
            [4] Row
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
                [1] Markdown(str)
            [5] ParamFunction(function, _pane=Column, defer_load=False)
        [3] Column
            [0] Select(name='Arena', options=['10: Hog Mountain', ...], value='10: Hog Mountain')
            [1] Markdown(str)
            [2] Row
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
                [1] ParamFunction(function, _pane=Column, defer_load=False)
        [4] Column
            [0] Select(name='Arena', options=['10: Hog Mountain', ...], value='10: Hog Mountain')
            [1] Row
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
            [2] Row
                [0] Markdown(str)
            [3] Row
                [0] Markdown(str)

# Main Results and Insights

## I was able to find insightful observations for the following:

## Cards:
Epic Cards are the most used cars throughout all arenas after Arena 3: Barbarian Bowl. Arena 3 is usually the Arena where players unlock the most powerful epic cards, which may be the reason for this observation. The usage of cards of other rarities keeps on varying in different arenas but epic cards always seem to be the most popular choice among players in all arenas. Before the analysis, I thought that legendary cards would be the most popular in higher arenas, but I was surprised to see the results. However, I can relate this observation to my actual gameplay. All players I play against (and me) use at least 3 epic cards on average in their decks so this observation checks out in real life.

## Elixir:
My assumptions about elixir actually checked out during the analysis. The average elixir stays pretty much the same throughout all arenas. Most players keep their deck at an elixir cost between 3.5 and 4.0. One interesting observation was that the average elixir seemed to climb a bit in the middle arenas (especially from Arena 6 onwards!) I think this is because these are the arenas in which you unlock legendary cards so many players use these new cards in their decks despite them costing a high amount of elixir. This behavior drives up the overall average. I did this myself when I was at these arenas in the game so I had a big grin on my face as soon as I saw this!
I also made four elixir clusters to compare different scenarios between winners and losers. The overall distribution seemed to stay the same between all arenas, however, the higher arenas had a lot fewer outliers. Most of the data points in higher arenas were closer to the median.

## Spells:
One question I often get asked by other players (mostly low level players) is: "Do spells matter?"
Personally, I never used any spell cards in my deck, and I never thought they made any difference. My analysis agreed with this assumption of mine for low arenas and higher arenas, but mid-range arenas told a different story. 
The ratio of winner and loser spells stayed the same for low and high arenas, but there was a significant difference in mid-level arenas. The difference started after Arena 4: Spell Valley. This is the arena in which a lot of new spells are unlocked (hence the arena name) so my guess is that a lot of players would experiment with different spells causing divergence in the overall trend.

## Crowns:
One interesting thing I observed in this is that as you go up in arena, the average number of crowns decreases. Lower arenas have much higher averages than higher arenas. This matches with my real-life experience as well. It is much easier to get 3 crown wins in lower arenas, but as you move to higher arenas, it becomes more difficult to get 3 crowns, hence most battles end in one crown wins. I used to get many 3 crown wins when I was in lower arenas, but now I mostly get 1 crown wins.

# Conclusion

The analysis was able to provide valuable insights ranging from user behavior to elixir trends. These results not only validated some of my preconceptions about the game but also helped me improve my strategy based on this newfound data. I hope it serves the same purpose to other people who play this game. Overall, this project was a satisfying experience as I got the opportunity to do an in-depth analysis of a topic of my interest.
